In [1]:
import requests
import datetime
import numpy as np
import pandas as pd

In [2]:
# Make a category lookup
def climbing_category(cat, intext=1):
    external_categories = {
        '1': 'menLead',
        '2': 'womenLead',
        '5': 'womenBouldering',
        '6': 'menBouldering',
        '23': 'menSpeed',
        '24': 'womenSpeed'
    }
    df_categories = {
        '1': 'Men',
        '2': 'Women',
        '5': 'Women',
        '6': 'Men',
        '23': 'Men',
        '24': 'Women'
    }
    return df_categories.get(cat, 'Men') if intext == 1 else external_categories.get(cat, 'otherCategory')

In [3]:
# Get overall information for all competitions in a season
def add_season_competitions(url_json):
    new_rows = []
    for comp in url_json['competitions']:
        new_comp = dict()
        
        # Add a dictionary with the information for each comp
        new_comp.update({
            'compId': comp.get('WetId'),
            'longCompName': comp.get('name'),
            'shortCompName': comp.get('short'),
            'hostNation': comp.get('host_nation'),
            'openComp': comp.get('open_comp'),
            'dateStart': comp.get('date'),
            'dateEnd': comp.get('date_end')
        })

        # Get all categories in this competition
        for cat in comp.get('cats'):
            new_comp.update({climbing_category(cat['GrpId'],2): 1})

        # Append new row dictionary, (representing a comp), in the new_rows list
        new_rows.append(new_comp)  
    
    # Return all new rows for the season
    return new_rows

In [4]:
# Today's date
today = datetime.datetime.now()

# Creating a competition overview dataframe
competition_overview_df = pd.DataFrame()

# Generate the link for all WCs and get all comp ids
for season in range(int(str(today.year)[-2:]) + 1):
    
    # Generating URLs for world cups since 2000 until this year
    url = 'http://egw.ifsc-climbing.org/egw/ranking/json.php?type=event&year='
    url = url + str(today.year - season)
    season_json = requests.get(url).json()
    
    # Getting all information for a dataframe with all comp overall figures
    competition_overview_df = competition_overview_df.append(add_season_competitions(season_json), ignore_index=True)
    
    # Adding what type of competitions are taking place
    competition_overview_df['isBoulderingComp'] = np.where(
        (competition_overview_df['menBouldering'] == 1) | 
        (competition_overview_df['womenBouldering'] == 1), 1, 0)
    
    competition_overview_df['isLeadComp'] = np.where(
        (competition_overview_df['menLead'] == 1) | 
        (competition_overview_df['womenLead'] == 1), 1, 0)
    
    competition_overview_df['isSpeedComp'] = np.where(
        (competition_overview_df['menSpeed'] == 1) | 
        (competition_overview_df['womenSpeed'] == 1), 1, 0)
    
competition_overview_df

E:\Data\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,compId,dateEnd,dateStart,hostNation,longCompName,menBouldering,menLead,menSpeed,openComp,otherCategory,shortCompName,womenBouldering,womenLead,womenSpeed,isBoulderingComp,isLeadComp,isSpeedComp
0,8328,2019-03-10,2019-03-09,HKG,Asian Cup (B) - Hong Kong (HKG) 2019,1.0,NaN,NaN,0,1.0,Asia_Cup_18,1.0,NaN,NaN,1,0,0
1,7906,2019-03-24,2019-03-23,None,International Climbing Series - Studio Bloc Ma...,NaN,NaN,NaN,3,NaN,Studio Bloc Masters,NaN,NaN,NaN,0,0,0
2,7903,2019-03-31,2019-03-30,None,Promo Event - Delfts Bleau International Youth...,NaN,NaN,NaN,0,NaN,Promo Delft,NaN,NaN,NaN,0,0,0
3,7923,2019-04-06,2019-04-05,SUI,IFSC Climbing Worldcup (B) - Meiringen (SUI) 2...,1.0,NaN,NaN,0,1.0,MeiringenWC,1.0,NaN,NaN,1,0,0
4,7926,2019-04-14,2019-04-12,RUS,"IFSC Climbing Worldcup (B,S) - Moscow (RUS) 2019",1.0,NaN,1.0,0,1.0,MoscowWC,1.0,NaN,1.0,1,0,1
5,8410,2019-04-21,2019-04-19,None,International Climbing Series - FISE World Ser...,NaN,NaN,NaN,3,NaN,FISE Hiroshima 2019,NaN,NaN,NaN,0,0,0
6,7929,2019-04-28,2019-04-26,CHN,"IFSC Climbing Worldcup (B,S) - Chongqing (CHN)...",1.0,NaN,1.0,0,1.0,ChongqingWC,1.0,NaN,1.0,1,0,1
7,7986,2019-04-28,2019-04-27,POR,European Youth Cup (B) - Soure (POR) 2019,NaN,NaN,NaN,3,1.0,Soure,NaN,NaN,NaN,0,0,0
8,8316,2019-05-04,2019-05-02,AUT,"IFSC European Cup (B,S) - Innsbruck (AUT) 2019",1.0,NaN,1.0,0,1.0,Inns19,1.0,NaN,1.0,1,0,1
9,7959,2019-05-05,2019-05-03,CHN,"IFSC Climbing Worldcup (B,S) - Wujiang (CHN) 2019",1.0,NaN,1.0,0,1.0,WujiangWC,1.0,NaN,1.0,1,0,1
